## Load Environments

In [1]:
from controlling_capital import *

Using TensorFlow backend.
/usr/local/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:212: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  arg_spec = inspect.getargspec(func)
/usr/local/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:212: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  arg_spec = inspect.getargspec(func)
/usr/local/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:212: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  arg_spec = inspect.getargspec(func)
/usr/local/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:212: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  arg_spec = inspect.getargspec(func)
/usr/local/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:212: DeprecationWarning: inspect.getargspec() is deprecated, use 

In [2]:
from __future__ import print_function
from __future__ import division
import warnings; warnings.filterwarnings('ignore')

import time
import random
from math import log

import numpy as np
import pandas as pd

random.seed(125)
np.random.seed(137)

In [3]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut, ParameterGrid
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_approximation import RBFSampler

In [4]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from xgboost import XGBClassifier

In [5]:
from numerapi.numerapi import NumerAPI
from sklearn.externals import joblib

In [6]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout

## Read in raw data sets, training, tournament (val, test, live)

In [7]:
x_train, y_train, train_eras, x_val, y_val, val_eras, x_test, x_live, feature_cols = import_data_sets()

## Kernal Approximation with SGD Classifier

In [9]:
#sk_sgd_model = joblib.load('saved/sk_sgd_model1514842414.pkl') 
#sk_sgd_grid = get_spacekitty_sgd_pipe()

In [ ]:
pipe = Pipeline([
        ('scaler', MinMaxScaler()),
        ('rbf', RBFSampler(random_state=73)),
        ('classify', SGDClassifier(loss='log', penalty='l2'))
    ])

    N_COMPONENTS = [7]
    GAMMA = [0.1]
    ALPHA = [0.001]

    param_grid = [{
            'rbf__n_components': N_COMPONENTS,
            'rbf__gamma': GAMMA,
            'classify__alpha': ALPHA
        }]

    sgd_grid = GridSearchCV(pipe, cv=5, n_jobs=1,
                           param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [10]:
sgd_grid.fit(x_train.values, y_train.values, groups=train_eras)
sgd_model = sgd_grid.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1 ......
[CV]  classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1, total=   1.0s
[CV] classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1 ......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV]  classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1, total=   0.9s
[CV] classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1 ......
[CV]  classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1, total=   0.9s
[CV] classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1 ......
[CV]  classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1, total=   0.9s
[CV] classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1 ......
[CV]  classify__alpha=0.001, rbf__n_components=7, rbf__gamma=0.1, total=   0.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.6s finished


In [12]:
print(sgd_grid.best_params_)
print(sgd_grid.best_score_)
calc_consistency(y_val.values, sgd_model.predict_proba(x_val.values)[:,1], val_eras)

{'classify__alpha': 0.001, 'rbf__n_components': 7, 'rbf__gamma': 0.1}
-0.693006450935
0.692557508771
0.691990070259
0.692824055698
0.69285240619
0.692776663778
0.692761880724
0.694008031638
0.692770166478
0.692326830812
0.693271003841
0.692210000157
0.692655484857
83.33333333333334


In [15]:
#joblib.dump(sgd_model, 'saved/sk_sgd_model' + str(int(time.time())) + '.pkl')

['saved/sk_sgd_model1514842414.pkl']

## Keras NN Model

In [16]:
#sk_nn_model = load_model('saved/sk_nn_model.h5')
#sk_nn_model = get_spacekitty_nn_model(x_train.values.shape[1])

In [ ]:
nn_model = Sequential()
nn_model.add(Dense(2048, activation='relu', input_dim=input_size))
nn_model.add(Dense(2048, activation='relu'))
nn_model.add(Dense(128, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(optimizer='rmsprop', loss='binary_crossentropy')

In [ ]:
nn_model.fit(x_train.values, y_train.values, epochs=50, batch_size=256,
             verbose=2, validation_data=(x_val.values, y_val.values))

In [17]:
calc_consistency(y_val.values, nn_model.predict(x_val.values)[:,0], val_eras)

0.693187226658
0.69185995405
0.692754740139
0.691465966345
0.692912073941
0.691386033033
0.69422584474
0.692626403434
0.690243344033
0.692883365658
0.692701965481
0.694108005542
75.0


In [18]:
#nn_model.save('saved/sk_nn_model' + str(int(time.time())) + '.h5')

## GridSearch CV Pipeline - XGBoost Classifier

In [8]:
#xgb_model = joblib.load('saved/sk_xgb_model1514843265.pkl') 
xgb_grid = get_spacekitty_xgb_pipe()

In [ ]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classify', XGBClassifier(eval_metric='logloss', n_jobs=-1, silent=False))
])

N_EST = [60]
LR = [0.01]
DEPTH = [10]

param_grid = [{
        'classify__n_estimators': N_EST,
        'classify__learning_rate': LR,
        'classify__max_depth': DEPTH        
    }]

xgb_grid = GridSearchCV(pipe, cv=5, n_jobs=1,
                        param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [9]:
xgb_grid.fit(x_train.values, y_train.values, groups=train_eras)
xgb_model = xgb_grid.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  53.3s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   54.0s remaining:    0.0s


[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  53.7s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  53.5s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  53.8s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  54.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.5min finished


In [10]:
print(xgb_grid.best_params_)
print(xgb_grid.best_score_)
calc_consistency(y_val.values, xgb_model.predict_proba(x_val.values)[:,1], val_eras)

{'classify__n_estimators': 60, 'classify__max_depth': 10, 'classify__learning_rate': 0.01}
-0.692990104896
0.692757578673
0.692333242709
0.692659891456
0.692046344997
0.692832007653
0.692471211325
0.693559660013
0.693367822003
0.692296648975
0.692565317921
0.693395092986
0.693128827844
75.0


In [12]:
joblib.dump(xgb_model, 'saved/sk_xgb_model' + str(int(time.time())) + '.pkl')

['saved/sk_xgb_model1514843265.pkl']

## Random Forest Model

In [13]:
#sk_rf_model = joblib.load('saved/sk_rf_model1514843488.pkl') 
#rf_grid = get_spacekitty_rf_pipe()

In [14]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('reduce_dim', PCA()),
    ('classify', RandomForestClassifier(n_jobs=-1, verbose=2, random_state=73))
])

N_DIM = [10]
N_EST = [50]
MAX_DEPTH = [5]

param_grid = [{
        'reduce_dim__n_components': N_DIM,
        'classify__n_estimators': N_EST,
        'classify__max_depth': MAX_DEPTH
    }]

rf_grid = GridSearchCV(pipe, cv=5, n_jobs=1,
                        param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [15]:
rf_grid.fit(x_train.values, y_train.values, groups=train_eras)
rf_model = rf_grid.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] classify__n_estimators=50, reduce_dim__n_components=10, classify__max_depth=5 
building tree 1 of 50building tree 2 of 50building tree 3 of 50building tree 4 of 50building tree 5 of 50




building tree 6 of 50
building tree 7 of 50building tree 8 of 50

building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.4s


building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    5.8s finished
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.2s finished


[CV]  classify__n_estimators=50, reduce_dim__n_components=10, classify__max_depth=5, total=   8.7s
[CV] classify__n_estimators=50, reduce_dim__n_components=10, classify__max_depth=5 


...

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    5.7s finished
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   45.9s finished


[CV]  classify__n_estimators=50, reduce_dim__n_components=10, classify__max_depth=5, total=   8.5s
building tree 2 of 50building tree 3 of 50
building tree 1 of 50building tree 4 of 50building tree 5 of 50building tree 6 of 50building tree 7 of 50





building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50building tree 18 of 50

building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.5s


building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    7.6s finished


In [16]:
print(rf_grid.best_params_)
print(rf_grid.best_score_)
calc_consistency(y_val.values, rf_model.predict_proba(x_val.values)[:,1], val_eras)

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


{'classify__n_estimators': 50, 'reduce_dim__n_components': 10, 'classify__max_depth': 5}
-0.692818238993
0.692189277096
0.691374326404
0.693032459618
0.692728632807
0.692099751276
0.692738940848
0.693301393094
0.692888588716
0.691372154566
0.692663005244
0.692807660105
0.694040364356
83.33333333333334


In [17]:
#joblib.dump(rf_model, 'saved/sk_rf_model' + str(int(time.time())) + '.pkl')

['saved/sk_rf_model1514843488.pkl']

## Ensemble

In [ ]:
to_pred = pd.concat([x_val, x_test, x_live])

In [ ]:
xgb_preds = pd.Series(xgb_model.predict_proba(to_pred.values)[:,1])
sgd_preds = pd.Series(sgd_model.predict_proba(to_pred.values)[:,1])
rf_preds = pd.Series(rf_model.predict_proba(to_pred.values)[:,1])
nn_preds = pd.Series(nn_model.predict(to_pred.values)[:,0])

to_vote = pd.concat([nn_preds, sgd_preds, rf_preds, xgb_preds], axis=1)
to_vote.columns = ['nn', 'sgd', 'rf', 'xgb']

# Ensemble Tree
ens_mean = to_vote.mean(axis=1)

In [ ]:
ens_pred = to_vote.apply(func=apply_vote,axis=1)

In [ ]:
ens_preds = ens_mean

In [ ]:
sub = pd.concat([pd.Series(to_pred.index), pd.Series(ens_preds)], axis=1)
sub.columns = ['id', 'probability']
sub.head()

In [ ]:
pred_path = 'sub_sk_88_' + str(int(time.time())) + '.csv'
sub.to_csv(pred_path, index=False)

In [ ]:
def get_model_key(model):
    if(model == "spacekitty"):
        return('MA2GHSBITR6I46CNQEX3K5K6YC7S4ET2','Z24UX4GGMJ27ZS3DN4PHMMYYQNBHCSSRIU7YB36KJ3DA5QDDRW37BQ3QD2ENID4D')
    elif(model=="wicker"):
        return('5KQJR4XR3DL7LJIYPI67JV377EXODJMQ','XRFADSYBTEH5TQREGP554QJ4VZGJXU5CGZOK6GPKCT4NCTDJ64T5M3URPL67JEY5')

In [ ]:
def upload_preds(path_to_preds, pub, priv):
    # set example username and round
    example_public_id = pub
    example_secret_key = priv

    # some API calls do not require logging in
    napi = NumerAPI(verbosity="info")
    
    # download current dataset
    napi.download_current_dataset(unzip=True)
    
    # provide api tokens
    napi = NumerAPI(example_public_id, example_secret_key)

    # upload predictions
    submission_id = napi.upload_predictions(path_to_preds)

In [ ]:
#pub, priv = get_model_key('spacekitty')
pub, priv = get_model_key('wicker')
upload_preds(pred_path, pub, priv)